<a href="https://colab.research.google.com/github/Hernanros/SOTA/blob/master/notebooks/labels/rouge_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install wmd

     |████████████████████████████████| 112kB 2.7MB/s 
  Created wheel for wmd: filename=wmd-1.3.2-cp36-cp36m-linux_x86_64.whl size=629397 sha256=e7f52dec50a1e6425db2700d50bc5a30adc1d4847d8dc0303e34ce83d18503d7
  Stored in directory: /root/.cache/pip/wheels/a6/4d/7a/fc3fdbc12e33c0551301543e4441fb4783c809a30451387ffd
Successfully built wmd


In [4]:
!pip install wordfreq
!git clone https://github.com/brmson/dataset-sts
# !wget  http://nlp.stanford.edu/data/glove.840B.300d.zip

     |████████████████████████████████| 32.8MB 120kB/s 
     |████████████████████████████████| 4.9MB 41.5MB/s 
     |████████████████████████████████| 276kB 41.9MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=952504f69ffa3a22b406cae47ed4b25da31c75af1f836bb21acffbdf84311d63
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=7416de6f8f2b25e6b7d3c53b2974de25785b666f437834c5603fb75caa42bb94
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862305 sha256=c5cdf6df07c72e286e81fda74bfda0cf282a9ae902bc9d6ef6f2aa12a38abbf7
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built wor

In [5]:
% cd /content/drive/My Drive/NLP_Project

/content/drive/My Drive/NLP_Project


In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from wmd import WMD
import wordfreq
import pickle

In [7]:
zip_ref = zipfile.ZipFile("./glove.840B.300d.zip")
zip_ref.extractall("/tmp")
zip_ref.close()

In [9]:
glove_pd = pd.read_csv('glove_pd.csv',index_col=0)
#glove_pd = pd.read_csv(open('glove.840B.300d.txt'), sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_pd.T.items()}
#glove_pd.to_csv('/content/drive/My Drive/NLP_Project/glove_pd.csv')
# np.savetxt(r'/content/drive/My Drive/NLP_Project/glove.txt', glove_pd.values, fmt='%d', delimiter=' ')

In [17]:
with open('combined_data.pickle', 'rb') as handle:
    df = pickle.load(handle)


In [21]:
tokenizer = RegexpTokenizer(r'\w+')
tokenizer.tokenize(df.iloc[9,0])
df['text_1_tokens'] =df.text_1.apply(lambda x:tokenizer.tokenize(x)) 
df['text_2_tokens'] =df.text_2.apply(lambda x:tokenizer.tokenize(x)) 

def embed (sent):
  toklist = tokenizer.tokenize(sent)
  embs = []
  freqs = []
  for word in toklist:
    try: 
      embs.append(glove[word.lower()])
      freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))
    except:
      try:
        embs.append(glove[word.strip("'")])
        freqs.append(wordfreq.word_frequency(word, 'en', wordlist='large'))

      except:
        continue
  return embs, freqs

embed(df.iloc[9,0])
text_1_embeddings = df.text_1.apply(lambda x: embed(x) )
text_2_embeddings = df.text_2.apply(lambda x: embed(x) )

# ROUGE similarities

In [36]:
!pip install easy-rouge

In [37]:
from rouge.rouge import rouge_n_sentence_level
from rouge.rouge import rouge_l_sentence_level

In [38]:
#sample
summary_sentence = 'the capital of China is Beijing'.split()
reference_sentence = 'Beijing is the capital of China'.split()

# Calculate ROUGE-2.
recall, precision, rouge = rouge_n_sentence_level(summary_sentence, reference_sentence, 2)
print('ROUGE-2-R', recall)
print('ROUGE-2-P', precision)
print('ROUGE-2-F', rouge)

recall, precision, rouge = rouge_l_sentence_level(summary_sentence, reference_sentence)
print('ROUGE-l-R', recall)
print('ROUGE-l-P', precision)
print('ROUGE-l-F', rouge)

ROUGE-2-R 0.6
ROUGE-2-P 0.6
ROUGE-2-F 0.6
ROUGE-l-R 0.6666666666666666
ROUGE-l-P 0.6666666666666666
ROUGE-l-F 0.6666666666666666


In [39]:

recall_1_list = []
recall_2_list = []
precision_1_list = []
precision_2_list = []
F_1_list = []
F_2_list = []

recall_l_list = []
precision_l_list = []
F_l_list = []

i = 0
while i < len(df):
    sentence_ori = df.iloc[i]['text_1']
    sentence_gen = df.iloc[i]['text_2']

    sentence_ori = sentence_ori.split()
    sentence_gen = sentence_gen.split()

    # Calculate ROUGE-1
    recall_1, precision_1, rouge_1 = rouge_n_sentence_level(sentence_gen, sentence_ori, 1)
    # Calculate ROUGE-2
    recall_2, precision_2, rouge_2 = rouge_n_sentence_level(sentence_gen, sentence_ori, 2)
    # Calculate ROUGE-l
    recall_l, precision_l, rouge_l = rouge_l_sentence_level(sentence_gen, sentence_ori)

    recall_1_list.append(recall_1)
    recall_2_list.append(recall_2)
    recall_l_list.append(recall_l)
    precision_1_list.append(precision_1)
    precision_2_list.append(precision_2)
    precision_l_list.append(precision_l)
    F_1_list.append(rouge_1)
    F_2_list.append(rouge_2)
    F_l_list.append(rouge_l)
    i+=1

In [40]:
df['ROUGE-1 recall'] = recall_1_list
df['ROUGE-1 precision'] = precision_1_list
df['ROUGE-1 F'] = F_1_list
df['ROUGE-2 recall'] = recall_2_list
df['ROUGE-2 precision'] = precision_2_list
df['ROUGE-2 F'] = F_2_list
df['ROUGE-L recall'] = recall_l_list
df['ROUGE-L precision'] = precision_l_list
df['ROUGE-L F'] = F_l_list

In [42]:
with open('combined_data.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)